In [1]:
!pwd

/home/micha/0_Dev/HACKATHON


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
df = pd.read_pickle('../data/conn_202011241955.pickle')
df_test = pd.read_csv('../data/UNSW-all.csv')

In [5]:
df_test.rename(columns={"spkts": "orig_pkts", "dpkts": "resp_pkts", "sbytes": "orig_ip_bytes", "dbytes": "resp_ip_bytes"}, errors="raise", inplace=True)

In [6]:
df_new = df_test

No.,Name,Type ,Description
* 7,dur,Float,Record total duration

* 8,sbytes,Integer,Source to destination transaction bytes (orig_ip_bytes)
* 9,dbytes,Integer,Destination to source transaction bytes (resp_ip_bytes)

* 15,Sload,Float,Source bits per second ((orig_ip_bytes/8) * orig_pkts)
* 16,Dload,Float,Destination bits per second ((resp_ip_bytes/8) * resp_pkts)

* 17,Spkts,integer,Source to destination packet count (orig_pkts)
* 18,Dpkts,integer,Destination to source packet count (resp_pkts)

* 23,smeansz,integer,Mean of the ?ow packet size transmitted by the src 
* 24,dmeansz,integer,Mean of the ?ow packet size transmitted by the dst

* 48,attack_cat,nominal,"The name of each attack category. In this data set , nine categories e.g. Fuzzers, Analysis, Backdoors, DoS Exploits, Generic, Reconnaissance, Shellcode and Worms"
* 49,Label,binary,0 for normal and 1 for attack records

In [7]:
del df_new['Unnamed: 0']
del df_new['id']

In [8]:
del df_new['ct_dst_sport_ltm']
del df_new['ct_dst_src_ltm']
del df_new['is_ftp_login']
del df_new['ct_ftp_cmd']
del df_new['ct_flw_http_mthd']
del df_new['ct_src_ltm']
del df_new['ct_srv_dst']
del df_new['is_sm_ips_ports']
del df_new['trans_depth']
del df_new['response_body_len']
del df_new['ct_srv_src']
del df_new['ct_state_ttl']
del df_new['ct_dst_ltm']
del df_new['ct_src_dport_ltm']	 	 	 
del df_new['proto']
del df_new['service']
del df_new['state']
del df_new['dwin']
del df_new['tcprtt']
del df_new['synack']
del df_new['ackdat']
del df_new['sloss']
del df_new['dloss']
del df_new['stcpb']
del df_new['dtcpb']
del df_new['smean'] #orig_ip_bytes / orig_pkts
del df_new['dmean'] #resp_ip_bytes / resp_pkts
del df_new['sttl']
del df_new['dttl']
del df_new['sload']
del df_new['dload']
del df_new['sinpkt']
del df_new['dinpkt']
del df_new['sjit']
del df_new['djit']
del df_new['swin']


In [9]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   dur            257673 non-null  float64
 1   orig_pkts      257673 non-null  int64  
 2   resp_pkts      257673 non-null  int64  
 3   orig_ip_bytes  257673 non-null  int64  
 4   resp_ip_bytes  257673 non-null  int64  
 5   rate           257673 non-null  float64
 6   attack_cat     257673 non-null  object 
 7   label          257673 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 15.7+ MB


In [10]:
df_new

,dur,orig_pkts,resp_pkts,orig_ip_bytes,resp_ip_bytes,rate,attack_cat,label
0,0.121478,6,4,258,172,74.087490,Normal,0
1,0.649902,14,38,734,42014,78.473372,Normal,0
2,1.623129,8,16,364,13186,14.170161,Normal,0
3,1.681642,12,12,628,770,13.677108,Normal,0
4,0.449454,10,6,534,268,33.373826,Normal,0
...,...,...,...,...,...,...,...,...
257668,0.000005,2,0,104,0,200000.005100,Normal,0
257669,1.106101,20,8,18062,354,24.410067,Normal,0
257670,0.000000,1,0,46,0,0.000000,Normal,0
257671,0.000000,1,0,46,0,0.000000,Normal,0


In [11]:
df

,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,duration,orig_bytes,resp_bytes
ts,,,,,,,,,,,,,,,,,,,,
2020-11-21 19:54:31,Cw4Ja82tfV4DIQxlak,62.171.150.88,32400,84.118.61.247,3839,tcp,,RSTRH,False,False,0,CaCCCCDfr,1,84,17556,702240,,175549,37972,0
2020-11-21 19:54:31,CjyTU14vvn2IJGZ2D6,87.123.141.161,57264,62.171.150.88,443,tcp,,OTH,False,False,0,^cADccTcTcT,54456,4956833,0,0,,22339.6,2.10855e+06,0
2020-11-21 19:54:31,CKtvM93sk5Dbyru6hk,87.123.141.161,32888,62.171.150.88,32400,tcp,,OTH,False,False,0,DcccAcTca,10091,607556,1,52,,22338.8,80424,281048
2020-11-21 19:54:31,CEQJy61S7lnsUBovC4,62.171.150.88,22888,87.123.141.161,50136,tcp,,OTH,False,False,0,CadCtCc,0,0,187,12244,,457.696,0,2564
2020-11-21 19:54:31,CgeI5t1Wyw11eN9GAc,87.123.141.161,52484,62.171.150.88,443,tcp,,RSTRH,False,False,0,DcAccr,219,17119,1,40,,370.979,5731,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-24 19:50:35,CZ5WOYyMv0ATAO8Xf,87.123.138.148,59454,62.171.150.88,443,tcp,ssl,S0,False,False,0,ScAD,7,1458,0,0,,0.144119,1086,0
2020-11-24 19:50:38,C8aRRT2JjdCMduHys,194.26.25.124,46546,62.171.150.88,33809,tcp,,REJ,False,False,0,SrR,2,80,1,40,,0.17907,0,0
2020-11-24 19:50:42,CgGHDa4EaXJeEPcB7,87.251.74.9,52716,62.171.150.88,1639,tcp,,REJ,False,False,0,SrR,2,80,1,40,,0.089327,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 475592 entries, 2020-11-21 19:54:31 to 2020-11-24 19:50:46
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   uid             475592 non-null  object
 1   id.orig_h       475592 non-null  object
 2   id.orig_p       475592 non-null  int64 
 3   id.resp_h       475592 non-null  object
 4   id.resp_p       475592 non-null  int64 
 5   proto           475592 non-null  object
 6   service         475592 non-null  object
 7   conn_state      475592 non-null  object
 8   local_orig      475592 non-null  bool  
 9   local_resp      475592 non-null  bool  
 10  missed_bytes    475592 non-null  int64 
 11  history         475592 non-null  object
 12  orig_pkts       475592 non-null  int64 
 13  orig_ip_bytes   475592 non-null  int64 
 14  resp_pkts       475592 non-null  int64 
 15  resp_ip_bytes   475592 non-null  int64 
 16  tunnel_parents  475592 non-null  object


# Columns bzw. names anpassen

In [13]:
df.reset_index(inplace=True)
del df['ts']
del df['uid']
del df['id.orig_h']
del df['id.orig_p']
del df['id.resp_h']
del df['id.resp_p']
del df['proto']
del df['service']
del df['tunnel_parents']
del df['history']
del df['local_orig']
del df['local_resp']
del df['conn_state']
del df['missed_bytes']



In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475592 entries, 0 to 475591
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   orig_pkts      475592 non-null  int64 
 1   orig_ip_bytes  475592 non-null  int64 
 2   resp_pkts      475592 non-null  int64 
 3   resp_ip_bytes  475592 non-null  int64 
 4   duration       475592 non-null  object
 5   orig_bytes     475592 non-null  object
 6   resp_bytes     475592 non-null  object
dtypes: int64(4), object(3)
memory usage: 25.4+ MB


In [15]:
df_david = df[['duration','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','orig_bytes','resp_bytes']]

In [16]:
df_david.reset_index(inplace=True)

In [17]:
df_david

,index,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes
0,0,175549,1,84,17556,702240,37972,0
1,1,22339.6,54456,4956833,0,0,2.10855e+06,0
2,2,22338.8,10091,607556,1,52,80424,281048
3,3,457.696,0,0,187,12244,0,2564
4,4,370.979,219,17119,1,40,5731,0
...,...,...,...,...,...,...,...,...
475587,475587,0.144119,7,1458,0,0,1086,0
475588,475588,0.17907,2,80,1,40,0,0
475589,475589,0.089327,2,80,1,40,0,0
475590,475590,0.359962,9,2192,0,0,1716,0


In [18]:
del df_david['index']

In [19]:
df_david

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes
0,175549,1,84,17556,702240,37972,0
1,22339.6,54456,4956833,0,0,2.10855e+06,0
2,22338.8,10091,607556,1,52,80424,281048
3,457.696,0,0,187,12244,0,2564
4,370.979,219,17119,1,40,5731,0
...,...,...,...,...,...,...,...
475587,0.144119,7,1458,0,0,1086,0
475588,0.17907,2,80,1,40,0,0
475589,0.089327,2,80,1,40,0,0
475590,0.359962,9,2192,0,0,1716,0


In [20]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   dur            257673 non-null  float64
 1   orig_pkts      257673 non-null  int64  
 2   resp_pkts      257673 non-null  int64  
 3   orig_ip_bytes  257673 non-null  int64  
 4   resp_ip_bytes  257673 non-null  int64  
 5   rate           257673 non-null  float64
 6   attack_cat     257673 non-null  object 
 7   label          257673 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 15.7+ MB


In [21]:
df_new.rename(columns={"dur": "duration"}, errors="raise", inplace=True)

In [22]:
df_new

,duration,orig_pkts,resp_pkts,orig_ip_bytes,resp_ip_bytes,rate,attack_cat,label
0,0.121478,6,4,258,172,74.087490,Normal,0
1,0.649902,14,38,734,42014,78.473372,Normal,0
2,1.623129,8,16,364,13186,14.170161,Normal,0
3,1.681642,12,12,628,770,13.677108,Normal,0
4,0.449454,10,6,534,268,33.373826,Normal,0
...,...,...,...,...,...,...,...,...
257668,0.000005,2,0,104,0,200000.005100,Normal,0
257669,1.106101,20,8,18062,354,24.410067,Normal,0
257670,0.000000,1,0,46,0,0.000000,Normal,0
257671,0.000000,1,0,46,0,0.000000,Normal,0


In [23]:
df_new = df_new[['duration','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','label']]
df_new.to_pickle('unsw-small.pickle')

In [24]:
#df_test.proto.unique()

In [25]:
df_new

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0.121478,6,258,4,172,0
1,0.649902,14,734,38,42014,0
2,1.623129,8,364,16,13186,0
3,1.681642,12,628,12,770,0
4,0.449454,10,534,6,268,0
...,...,...,...,...,...,...
257668,0.000005,2,104,0,0,0
257669,1.106101,20,18062,8,354,0
257670,0.000000,1,46,0,0,0
257671,0.000000,1,46,0,0,0


In [67]:
import joblib
#joblib.dump(rf, "./random_forest.joblib")
loaded_rf = joblib.load("../model/random_forest.joblib")

In [27]:
del df_david['orig_bytes']
del df_david['resp_bytes']

In [28]:
df_david.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475592 entries, 0 to 475591
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   duration       475592 non-null  object
 1   orig_pkts      475592 non-null  int64 
 2   orig_ip_bytes  475592 non-null  int64 
 3   resp_pkts      475592 non-null  int64 
 4   resp_ip_bytes  475592 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 18.1+ MB


In [29]:
df_david.to_pickle('../data/davidsSet.pickle')

In [28]:
#df_david['duration'] = pd.to_numeric(df['duration'],errors='coerce')
#df_david['duration'] = df_david['duration'].astype(float)
count = 0
df_david['duration'] = [0.0 if isinstance(x,str) else x for x in df_david['duration']]



<ipython-input-28-ff58c6b1b5da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_david['duration'] = [0.0 if isinstance(x,str) else x for x in df_david['duration']]


In [29]:
results = loaded_rf.predict(df_david)

In [30]:
df_david["prediction"] = 0.0

<ipython-input-30-cf133fa25b5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_david["prediction"] = 0.0


In [31]:
df["prediction"] = 0.0

In [32]:
for i in range(len(results)):
    df["prediction"][i] = results[i] 

<ipython-input-32-9a547dc4d379>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prediction"][i] = results[i]


In [33]:
df.to_pickle('connlog-with-prediction.pickle')

In [34]:
len(df_david)

475592

In [35]:
len(results)

475592

In [36]:
for i in range(len(results)):
    df_david["prediction"][i] = results[i] 

<ipython-input-36-e61857a9eb71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_david["prediction"][i] = results[i]
/home/micha/anaconda3/envs/hackathon2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
df_david.prediction.describe()

count    475592.000000
mean          0.142641
std           0.349707
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: prediction, dtype: float64

In [38]:
df_david[df_david.prediction>=0.99]

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,prediction
7,209.875541,151036,2485906334,181125,9429656,1.0
10,26411.548384,0,0,2642,137576,1.0
13,22334.079740,0,0,10066,589936,1.0
118,232.883177,171676,2825660852,201907,10507844,1.0
167,6.543828,883,103540,98,2150104,1.0
...,...,...,...,...,...,...
475463,55.150433,11,615,4,160,1.0
475465,55.077840,11,615,4,160,1.0
475505,54.953765,11,615,5,200,1.0
475513,54.856970,10,551,3,120,1.0


In [40]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [41]:
profile = ProfileReport(df_david, title='Prediction Profiling Report', minimal=True)

/home/micha/anaconda3/envs/hackathon2/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [42]:
type(profile)

pandas_profiling.profile_report.ProfileReport

In [43]:
profile.to_file("../PredictionProfiling.html")

In [ ]:
#profile = ProfileReport(df_test, title='TestSet Profiling Report')

In [ ]:
#profile.to_file("ProfileTest.html")